In [14]:
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go

In [15]:
def value_consistency(col, existing_values, unik_value, data):
    mask = data[col].isin(existing_values)
    #update
    data.loc[(mask), col] = unik_value


In [16]:
def clean_df(df_raw):
    # Venue
    value_consistency('venue',['M Chinnaswamy Stadium', 'M.Chinnaswamy Stadium'], 'M Chinnaswamy Stadium', df_raw)
    value_consistency('venue',['MA Chidambaram Stadium', 'MA Chidambaram Stadium, Chepauk',
                                 'MA Chidambaram Stadium, Chepauk, Chennai'], 'MA Chidambaram Stadium, Chepauk', df_raw)
    value_consistency('venue',['Punjab Cricket Association IS Bindra Stadium',
           'Punjab Cricket Association IS Bindra Stadium, Mohali',
           'Punjab Cricket Association Stadium, Mohali'], 'Punjab Cricket Association Stadium, Mohali', df_raw)
    value_consistency('venue',['Rajiv Gandhi International Stadium',
           'Rajiv Gandhi International Stadium, Uppal'], 'Rajiv Gandhi International Stadium, Uppal', df_raw)
    value_consistency('venue',['Wankhede Stadium', 'Wankhede Stadium, Mumbai'], 'Wankhede Stadium', df_raw)
    
    df_raw.season = df_raw.season.map(str)
    
    value_consistency('batting_team', ['Delhi Capitals', 'Delhi Daredevils'], 'Delhi Capitals', df_raw)
    value_consistency('batting_team', ['Pune Warriors','Rising Pune Supergiant', 'Rising Pune Supergiants'], 'Rising Pune Supergiant', df_raw)

    value_consistency('bowling_team', ['Delhi Capitals', 'Delhi Daredevils'], 'Delhi Capitals', df_raw)
    value_consistency('bowling_team', ['Pune Warriors','Rising Pune Supergiant', 'Rising Pune Supergiants'], 'Rising Pune Supergiant', df_raw)
    
    

In [17]:
# Batting metrics calculation
def count_4s(c):
    return ((c == 4) | (c==5)).sum()

def count_6s(c):
    return (c == 6).sum()

def get_balls_faced(c):
    return c.shape[0]

def get_wide_balls(c):
    return int(c.notnull().sum())

def get_total_runs(c):
    return c.sum()

def get_if_century(c):
    return int(c.sum() >= 100)

def get_if_half_century(c):
    return int(50 <= c.sum() < 100)

In [18]:
def bowler_wickets(s):
    return (s.notnull() & s.isin(['caught', 'bowled', 'lbw', 'stumped', 'caught and bowled', 'hit wicket'])).sum()

def get_if_3knocked(s):
    w = bowler_wickets(s)
    return int(w == 3)
    
def get_if_4knocked(s):
    w = bowler_wickets(s)
    return int(3 < w)

def get_if_shortwickets(s):
    w = bowler_wickets(s)
    return int(w < 3)

In [19]:
def transformFile(filename):
    df = pd.read_csv('dataset/individual_match/'+filename)
    
    # Clean
    clean_df(df)
    
    batsman_stats = df.groupby(['match_id', 'season', 'venue', 'innings','striker']).agg(
        no_of_4s = ('runs_off_bat', count_4s),
        no_of_6s = ('runs_off_bat', count_6s),
        balls_faced = ('ball', get_balls_faced), # faced = played + wides
        total_wides = ('wides', get_wide_balls),
        total_runs = ('runs_off_bat', get_total_runs),
        is_century = ('runs_off_bat', get_if_century),
        is_half_century = ('runs_off_bat', get_if_half_century)
    ).reset_index()
    batsman_stats['balls_faced'] = batsman_stats['balls_faced'].astype('int')
    batsman_stats['total_wides'] = batsman_stats['total_wides'].astype('int')
        
    batsman_stats['balls_played'] = batsman_stats.balls_faced - batsman_stats.total_wides
    batsman_stats['hard_hit_rate'] = (batsman_stats.no_of_4s + batsman_stats.no_of_6s) / batsman_stats.balls_played
    batsman_stats['fast_score_rate'] = (batsman_stats.total_runs / batsman_stats.balls_played)
    batsman_stats['run_bw_wickets'] = (batsman_stats.total_runs - (batsman_stats.no_of_4s*4 + batsman_stats.no_of_6s*6)) / (batsman_stats.balls_played - (batsman_stats.no_of_4s + batsman_stats.no_of_6s))
    
    batsman_stats_to_keep = ['match_id', 'season', 'venue', 'innings', 'striker','is_century','is_half_century','run_bw_wickets', 'hard_hit_rate','fast_score_rate', 'balls_played']
    batsman_stats = batsman_stats.loc[:, batsman_stats_to_keep]
#     print('batsman_stats',filename,batsman_stats.shape)
    
    # Bowler
    bowler_stats = df.groupby(['match_id', 'season', 'venue', 'innings','bowler']).agg(
        runs_scored_bat = ('runs_off_bat', 'sum'),
        runs_scored_extra = ('extras', 'sum'),
        total_wides = ('wides', 'count'),
        total_noballs = ('noballs', 'count'),
        total_balls = ('ball', 'count'),
        wickets_taken = ('wicket_type', bowler_wickets),
        wiskets_3plus = ('wicket_type', get_if_3knocked),
        wickets_4plus = ('wicket_type', get_if_4knocked),
        wickets_shortIndex = ('wicket_type', get_if_shortwickets)
    ).reset_index()

    bowler_stats['economy'] = (bowler_stats.runs_scored_bat + bowler_stats.runs_scored_extra) / (bowler_stats.total_balls // 6)
    bowler_stats['strike_rate'] = bowler_stats.total_balls / bowler_stats.wickets_taken
    bowler_stats['bowling_average'] = (bowler_stats.runs_scored_bat + bowler_stats.runs_scored_extra) / bowler_stats.wickets_taken
    bowler_stats['illegal_bowl_rate'] = (bowler_stats.total_wides + bowler_stats.total_noballs) / bowler_stats.total_balls

    bowler_stats_to_keep =  ['match_id', 'season', 'venue', 'innings', 'bowler','illegal_bowl_rate','wiskets_3plus','wickets_4plus','wickets_shortIndex','economy','strike_rate', 'bowling_average']
    bowler_stats = bowler_stats.loc[:, bowler_stats_to_keep]
#     print('bowler_stats',filename,bowler_stats.shape)
    
    
    batsman_stats.to_csv('dataset/individual_match_m1/mod_batsman_stats_'+filename, index=False, na_rep="NaN")
    bowler_stats.to_csv('dataset/individual_match_m1/mod_bowler_stats_'+filename, index=False, na_rep="NaN")
    
    
    

## Actual Transformation

In [20]:
# assert False
import os
input_files = os.listdir('dataset/individual_match')
count = 0
for fileName in input_files:
    try:
        transformFile(fileName)
        f_reload = pd.read_csv('dataset/individual_match_m1/mod_batsman_stats_'+fileName, index_col=None, header=0)
        f_reload = pd.read_csv('dataset/individual_match_m1/mod_bowler_stats_'+fileName, index_col=None, header=0)
    except RuntimeError as e:
        print(e)
        count += 1
print("Total files failed ", count)

Total files failed  0


### Merge all into One

In [21]:
# Merging all batsman
import os
mod_files = os.listdir('dataset/individual_match_m1')
li = []
for filename in mod_files:
    if('batsman' not in filename):
        continue
    df = pd.read_csv('dataset/individual_match_m1/'+filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

(12269, 11)


In [22]:
frame.to_csv('dataset/all_matches_m1_batsman.csv', index=False, na_rep="NaN")

In [23]:
# Merging all batsman
# assert False
import os
mod_files = os.listdir('dataset/individual_match_m1')
li = []
for filename in mod_files:
    if('bowler' not in filename):
        continue
    df = pd.read_csv('dataset/individual_match_m1/'+filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)
print(frame.shape)

(9731, 12)


In [24]:
frame.to_csv('dataset/all_matches_m1_bowler.csv', index=False, na_rep="NaN")

## Final Data Population

In [29]:
df = pd.read_csv('dataset/all_matches.csv')
print(df.shape)
df.columns

(194354, 22)


Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [43]:
striker_bowler = df.groupby(['venue', 'innings', 'match_id','season', 'striker', 'bowler']).agg(
    balls = ('ball', 'count'),
    runs_bat = ('runs_off_bat', 'sum'),
    extras = ('extras', 'sum'),
).reset_index().sort_values(by='match_id')
striker_bowler[:1]

,venue,innings,match_id,season,striker,bowler,balls,runs_bat,extras
15554,M Chinnaswamy Stadium,2,335982,2007/08,JH Kallis,AB Agarkar,4,7,0


In [45]:
bowler_stats = pd.read_csv('dataset/all_matches_m1_bowler.csv')
batsman_stats = pd.read_csv('dataset/all_matches_m1_batsman.csv')

In [87]:
b = batsman_stats.query('striker == "{0}" and season == "{1}" and match_id < {2}'
                        .format("V Kohli",'2009', 392218)).drop(
    ['striker','match_id','venue','season','innings', 'balls_played'], axis=1)
b

,is_century,is_half_century,run_bw_wickets,hard_hit_rate,fast_score_rate
1710,0,0,0.375000,0.000000,0.375000
1937,0,0,0.666667,0.000000,0.666667
3146,0,0,0.416667,0.076923,0.846154
4263,0,0,0.900000,0.166667,1.583333
5077,0,0,0.523810,0.045455,0.681818
6176,0,1,0.880000,0.218750,1.562500
8625,0,0,0.666667,0.142857,1.142857
8745,0,0,0.782609,0.041667,0.916667


In [79]:
for row in striker_bowler[:1].iterrows():
    rf = row[1]
    print(rf)
    
    bt = batsman_stats.query('striker == "{0}" and season == "{1}" and match_id < {2}'
                             .format(rf.striker, rf.season, rf.match_id)).mean()
    bo = bowler_stats.query('bowler == "{0}" and season == "{1}" and match_id < {2}'
                             .format(rf.bowler, rf.season, rf.match_id)).mean()
    
    rf = rf.append(b[3:-1])
    print(rf)

venue       M Chinnaswamy Stadium
innings                         2
match_id                   335982
season                    2007/08
striker                 JH Kallis
bowler                 AB Agarkar
balls                           4
runs_bat                        7
extras                          0
Name: 15554, dtype: object
venue              M Chinnaswamy Stadium
innings                                2
match_id                          335982
season                           2007/08
striker                        JH Kallis
bowler                        AB Agarkar
balls                                  4
runs_bat                               7
extras                                 0
is_century                             0
is_half_century                    0.125
run_bw_wickets                  0.651427
hard_hit_rate                  0.0865398
fast_score_rate                 0.971874
dtype: object


In [70]:
b.vstack(b)

AttributeError: 'DataFrame' object has no attribute 'vstack'